# **Autoencoder denoiser**

### A CNN to denoise corrupted images based on Fashion MNIST dataset.

Autoencoder architecture provides an encoded latent space of reduced features which can be decoded back to its original form.

Autoencoder is an unsupervised methodology similar, but more flexible than Principal Component Analysis (PCA). It doesn't resemble the probabilistic distribution unlike Variational Auto Encoders.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

## 1. Dataset

### 1.1 Load the dataset
In this case we are loading dataset separately for train and test dataset.

We include an artificial noise to the image.

In [ ]:
train_dataset, test_dataset = tfds.load('fashion_mnist', as_supervised=True, split=['train', 'test'])

### 1.2 Normalize and add noise
- Normalize the values to the range of 0-1
- Intentionally corrupt the dataset with random noise.
- We want a grainy pictures, not a slight noise.
- **Important: we replace the classification labels with original images. We want to compare images instead of classifying them.**

In [ ]:
def add_noise(image, label):
    image = tf.cast(image, dtype=tf.float32)
    image /= 255.0

    noise = tf.random.normal(shape=image.shape)
    noise *= 0.5

    noise_image = image + noise
    noise_image = tf.clip_by_value(noise_image, 0.0, 1.0)

    return noise_image, image

### 1.3 Process the images in datasets

In [ ]:
BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 1024

train_dataset = train_dataset.map(add_noise)
test_dataset = test_dataset.map(add_noise)

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE).repeat()
test_dataset = test_dataset.batch(BATCH_SIZE).repeat()

### 1.4 Preview the noisy images
*By the way -  brain processing according to gestalt principles and top-down visual perception mostly makes it trivial to guess what's on the picture - but some of them doesn't look like they contain anything at all. It's fascinating how AI can handle that.*

In [ ]:
plt.figure(figsize=(15, 5))

row = 1
for image, label in train_dataset.take(10):
    plt.subplot(1, 10, row)
    plt.imshow(image[0], cmap='gray')
    plt.axis('off')
    row += 1

## 2. Create the Model

**Autoencoder consists of 3 building blocks:**
- **Encoder:** Reduce dimensions to low-dimensional latent space.
- **Bottleneck:** The most compressed representation of our data.
- **Decoder:** Decodes the image from it's compressed form.

Instead of sequential or functional API, these building blocks are made as classes.

### 2.1 Encoder

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self):
        super(Encoder, self).__init__()
        self.conv_1 = tf.keras.layers.Conv2D(filters=64, 
                                             kernel_size=(3,3), 
                                             activation='relu', 
                                             padding='same')

        self.maxpool_1 = tf.keras.layers.MaxPooling2D(pool_size=(2,2))

        self.conv_2 = tf.keras.layers.Conv2D(filters=128, 
                                             kernel_size=(3,3), 
                                             activation='relu', 
                                             padding='same')

        self.maxpool_2 = tf.keras.layers.MaxPooling2D(pool_size=(2,2))


    def call(self, inputs):
        x = self.conv_1(inputs)
        x = self.maxpool_1(x)
        x = self.conv_2(x)
        x = self.maxpool_2(x)
        return x

### 2.2 Bottleneck

In [ ]:
class Bottleneck(tf.keras.Model):
    def __init__(self):
        super(Bottleneck, self).__init__()
        self.bottleneck = tf.keras.layers.Conv2D(filters=256, 
                                                 kernel_size=(3,3), 
                                                 activation='relu', 
                                                 padding='same')

        self.encoder_visualization = tf.keras.layers.Conv2D(filters=1, 
                                                            kernel_size=(3,3), 
                                                            name='encoder_visualization',
                                                            activation='sigmoid', 
                                                            trainable=False,
                                                            padding='same')

    def call(self, inputs):
        x = self.bottleneck(inputs)
        vis = self.encoder_visualization(x)
        return x, vis

### 2.3 Decoder

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self):
        super(Decoder, self).__init__()
        self.conv_1 = tf.keras.layers.Conv2D(filters=128, 
                                             kernel_size=(3,3), 
                                             activation='relu', 
                                             padding='same')
        
        self.upsample_1 = tf.keras.layers.UpSampling2D(size=(2,2))

        self.conv_2 = tf.keras.layers.Conv2D(filters=64, 
                                             kernel_size=(3,3), 
                                             activation='relu', 
                                             padding='same')
        
        self.upsample_2 = tf.keras.layers.UpSampling2D(size=(2,2))

        self.conv_3 = tf.keras.layers.Conv2D(filters=1, 
                                             kernel_size=(3,3), 
                                             activation='sigmoid', 
                                             padding='same')

    def call(self, inputs):
        x = self.conv_1(inputs)
        x = self.upsample_1(x)
        x = self.conv_2(x)
        x = self.upsample_2(x)  
        x = self.conv_3(x)
        return x

### 2.4 Autoencoder model
Merge each subclasses into the final Autoencoder model.

**Keras subclass API provides great flexibility but differers from functional and sequential API:**
- You can customize every function but bugs are harder to track.
- To build the model you have to pass it's initial input shape `RAW_INPUT_SHAPE`
- Model summary doesn't provide the output sizes unless you create a customized summary function which passes the input shape and then calls summary.

*Autoencoder might be too basic for this kind of development because of many boilerplate code necessary.*

**TODO: Custom compile and training loop**


In [ ]:
class AutoEncoder(tf.keras.Model):
    def __init__(self):
        super(AutoEncoder, self).__init__()

        self.encoder = Encoder()
        self.bottleneck = Bottleneck()
        self.decoder = Decoder()


    def call(self, inputs):

        x = self.encoder(inputs)
        x, _ = self.bottleneck(x)
        x = self.decoder(x)
        return x


    def summary(self, shape):
        x = tf.keras.layers.Input(shape=shape)
        temp_model = tf.keras.Model(inputs=x, outputs=self.call(x))
        return temp_model.summary()


INPUT_SHAPE = (28,28,1)
RAW_INPUT_SHAPE = (1, *INPUT_SHAPE)

denoise_model = AutoEncoder()
denoise_model.build(RAW_INPUT_SHAPE)
denoise_model.summary(INPUT_SHAPE)

# denoise_model.encoder.summary()
# denoise_model.bottleneck.summary()
# denoise_model.decoder.summary()

## 3. Compile and train

In [ ]:
TRAIN_STEPS = 60000 // BATCH_SIZE
VALID_STEPS = 60000 // BATCH_SIZE
EPOCHS = 40

denoise_model.compile(
    optimizer=tf.keras.optimizers.Adam(), 
    loss=tf.keras.losses.BinaryCrossentropy()
)

denoise_model_history = denoise_model.fit(train_dataset, steps_per_epoch=TRAIN_STEPS, validation_data=test_dataset, validation_steps=VALID_STEPS, epochs=EPOCHS)

## 4. Results

### 4.1 Visualization helper model
A helper model to reach the most compressed input representation inside the bottleneck. It's a subclass of Autoencoder but returns the helper layer instead of passing data to decoder.

In [ ]:
class Encoder_visualization(tf.keras.Model):
    def __init__(self):
        super(Encoder_visualization, self).__init__()
        self.encoder = denoise_model.encoder
        self.bottleneck = denoise_model.bottleneck
    

    def call(self, inputs):
        x = self.encoder(inputs)
        _, vis = self.bottleneck(x)

        return vis


encoder_visualization_model = Encoder_visualization()
encoder_visualization_model.build(RAW_INPUT_SHAPE)
encoder_visualization_model.summary()

### 4.2 Display denoising result

In [ ]:
def display_one_row(disp_images, offset, shape):
    for index, noisy_image in enumerate(disp_images):
        plt.subplot(3, 10, offset + index + 1)
        plt.xticks([])
        plt.yticks([])
        noisy_image = np.reshape(noisy_image, shape)
        plt.imshow(noisy_image, cmap='gray')


def display_results(disp_input_images, disp_encoded, disp_predicted, enc_shape):
    plt.figure(figsize=(15, 5))
    display_one_row(disp_input_images, 0, shape=(28,28))
    display_one_row(disp_encoded, 10, shape=enc_shape)
    display_one_row(disp_predicted, 20, shape=(28,28))

In [ ]:
TEST_BATCH = test_dataset.take(1)
PREVIEW_AMOUNT = 10
INDEX = np.random.randint(0, BATCH_SIZE, size=PREVIEW_AMOUNT)

output_samples = []
for input_image, image in tfds.as_numpy(TEST_BATCH):
    output_samples = input_image

output_samples = np.array(output_samples[INDEX])

encoded = encoder_visualization_model.predict(output_samples)
predicted = denoise_model.predict(output_samples)

display_results(output_samples, encoded, predicted, enc_shape=(7,7))